In [85]:
from models.nac import NAC
import torch.nn.functional as F
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm_notebook as tqdm

from data_generator_helper import generate_synthetic_selection_dataset

import math
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

from torch.nn.parameter import Parameter

In [86]:
from torch.nn import MSELoss as MSE

def test(model, x_test, y_test):
    loss = nn.MSELoss()
    model.eval()
    output_test = model(x_test)
    loss = loss(output_test, y_test)
    
    return loss

In [87]:
weightfile = 'convtest_NALU_Ones_test_3.npy'
weights, losses, bounds, sample_size, out_dim, epochs, its, g_prevs = np.load(weightfile)
temp = weights[0][0][-1,-1,:,:,:]
W_hat = temp[:,:,0]
M_hat = temp[:,:,1]
W = temp[:,:,2]
G = temp[:,:,3]


In [88]:
class NeuralAccumulatorCell(nn.Module):
    """A Neural Accumulator (NAC) cell [1].

    Attributes:
        in_dim: size of the input sample.
        out_dim: size of the output sample.

    Sources:
        [1]: https://arxiv.org/abs/1808.00508
    """
    def __init__(self, What, Mhat):
        super().__init__()


        self.What = What#Parameter(torch.Tensor(out_dim, in_dim))
        self.Mhat = Mhat#Parameter(torch.Tensor(out_dim, in_dim))

        self.register_parameter('W_hat', self.What)
        self.register_parameter('M_hat', self.Mhat)
        self.register_parameter('bias', None)

    def forward(self, input):
        W = torch.tanh(self.What) * torch.sigmoid(self.Mhat)
        return F.linear(input, W, self.bias)


class NeuralArithmeticLogicUnitCell(nn.Module):
    """A Neural Arithmetic Logic Unit (NALU) cell [1].

    Attributes:
        in_dim: size of the input sample.
        out_dim: size of the output sample.

    Sources:
        [1]: https://arxiv.org/abs/1808.00508
    """
    def __init__(self, G, What, Mhat):
        super().__init__()

        self.eps = 1e-10

        self.G = G
        self.nac = NeuralAccumulatorCell(What, Mhat)
        self.register_parameter('bias', None)
    

    def forward(self, input):

        a = self.nac(input)
        g = torch.sigmoid(F.linear(input, self.G, self.bias))

        add_sub = g * a
        log_input = torch.log(torch.abs(input) + self.eps)

        m = torch.exp(self.nac(log_input))

        mul_div = (1 - g) * m
        y = add_sub + mul_div
        return y


In [89]:
G[0,0],type(G[0,0])

(1.0, numpy.float64)

In [90]:
G = torch.Tensor(G)
G = torch.nn.Parameter(G.type(torch.DoubleTensor))
W_hat = torch.Tensor(W_hat)
W_hat = torch.nn.Parameter(W_hat.type(torch.DoubleTensor))
M_hat = torch.Tensor(M_hat)
M_hat = torch.nn.Parameter(M_hat.type(torch.DoubleTensor))

model = NeuralArithmeticLogicUnitCell(G, W_hat, M_hat)

In [91]:
np.shape(G), np.shape(W_hat), np.shape(M_hat)

(torch.Size([2, 100]), torch.Size([2, 100]), torch.Size([2, 100]))

In [92]:
test_per_range = 1
sample_size = 100
set_size = 100

in_dim = sample_size
hidden_dim = 1
out_dim = 2
num_layers = 1

lr = 0.01
epochs = 1000


#values = np.linspace(1,1000,10)
values = [[0.0,1.0],
          [-0.001, 0.001],
          [0.0,100.0],
          [-100.0,100.0],
          [-1000.0,1000.0],
          [-1.0,1.0]]

for value in values:
    min_value, max_value=  value

    x_test, y_test, _ = generate_synthetic_selection_dataset(min_value, max_value,
                                                                sample_size, set_size, boundaries = bounds[0][0])

    x_test = x_test.type(torch.DoubleTensor)
    y_test = y_test.type(torch.DoubleTensor)
    test_loss  = test(model, x_test, y_test)

    print(float(test_loss.data.numpy())/torch.max(x_test).numpy())


1.387712434366338e-08
0.0005664076238968713
1.0444664161538606e-06
6.126072214472623e+39
1.5438243626209182e+64
0.6881436360393461


In [133]:
weightfile = 'nntest_NALU_[\'NALU\']_test_22_50.npy'
out = np.load(weightfile)
data = out[1]
w1 = data[0][0].double()
b1 = data[0][1].double()
w2 = data[2][0].double()
b2 = data[2][1].double()

k1 = F.linear(x_test,w1)
k2 = k1+b1
k3 = F.linear(k2,w2)
k4 = k3+b2
v = 10
print(k4[v,:])
print(y_test[v,:])



tensor([13.4463,  4.6190], dtype=torch.float64, grad_fn=<SelectBackward>)
tensor([0.9024, 2.1096], dtype=torch.float64)


In [238]:
test_per_range = 1
sample_size = 100
set_size = 100

in_dim = sample_size
hidden_dim = 1
out_dim = 2
num_layers = 1

lr = 0.01
epochs = 1000

shit = ['nntest_NALU_[\'NALU\']_test_0_2_50_100.npy', 'nntest_NALU_[\'NALU\']_test_1_2_50_100.npy',
'nntest_NALU_[\'NALU\']_test_2_2_50_100.npy', 'nntest_NALU_[\'NALU\']_test_3_2_50_100.npy',
'nntest_NALU_[\'NALU\']_test_4_2_50_100.npy', 'nntest_NALU_[\'NALU\']_test_5_2_50_100.npy',
'nntest_NALU_[\'NALU\']_test_6_2_50_100.npy', 'nntest_NALU_[\'NALU\']_test_7_2_50_100.npy',
'nntest_NALU_[\'NALU\']_test_8_2_50_100.npy', 'nntest_NALU_[\'NALU\']_test_9_2_50_100.npy']
    
for weightfile in shit:
    #weightfile = 'nntest_NALU_[\'NALU\']_test_2_2_50_100.npy'
    print(weightfile)
    #weightfile = 'nntest_NALU_[\'NALU\']_test_02_50.npy'
    out = np.load(weightfile)
    data = out[1]
    bounds = out[2]
    w1 = data[0][0].double()
    b1 = data[0][1].double()
    w2 = data[2][0].double()
    b2 = data[2][1].double()


    #values = np.linspace(1,1000,10)
    values = [[0.0,1.0],
              [-0.001, 0.001],
              [0.0,100.0],
              [-100.0,100.0],
              [-1000.0,1000.0],
              [-1.0,1.0]]

    for value in values:
        min_value, max_value=  value

        x_test, y_test, _ = generate_synthetic_selection_dataset(min_value, max_value,
                                                                    sample_size, set_size, boundaries = bounds)

        x_test = x_test.type(torch.DoubleTensor)
        y_test = y_test.type(torch.DoubleTensor)

        loss = nn.MSELoss()

        k1 = F.linear(x_test,w1)
        k2 = F.relu(k1+b1)
        k3 = F.linear(k2,w2)
        k4 = F.relu(k3+b2)

        #print(np.shape(k4))
        #print(np.shape(y_test))
        test_loss = loss(k4,y_test)

        #test_loss  = test(model, x_test, y_test)
        print(float(test_loss.data.numpy())/torch.max(x_test).numpy())



nntest_NALU_['NALU']_test_0_2_50_100.npy
1.5908613926457666
25397.913736081275
1009.1401081566196
990.4139796171725
9759.784672545507
36.01126898297484
nntest_NALU_['NALU']_test_1_2_50_100.npy
0.8880948662311533
23436.821219501195
1801.133432992231
587.7638878711247
5919.64327899
31.66429423169649
nntest_NALU_['NALU']_test_2_2_50_100.npy
0.5431079555336887
22770.759352205627
1270.1476144566182
554.1233911547836
4800.181885365624
27.323166147510733
nntest_NALU_['NALU']_test_3_2_50_100.npy
0.4733716973777383
18005.088088650125
1377.3568646401695
480.322123496499
4974.480400071965
26.10149378691088
nntest_NALU_['NALU']_test_4_2_50_100.npy
3.0793538456534386
54187.334248876534
1551.5749116923785
1895.3682821870318
18211.243496942374
77.61918446588143
nntest_NALU_['NALU']_test_5_2_50_100.npy
1.183094937895295
94250.60812487072
2655.7503055652633
945.7321515843074
8928.981739551498
96.59777629458857
nntest_NALU_['NALU']_test_6_2_50_100.npy
3.803165920908013
180466.09711344176
4171.3352384967